**load the dataset**

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

def get_examples(dataset_name, val_size):
    # load dataset
    dataset = load_dataset(dataset_name)

    # access the validation dataset
    valid_data = dataset['validation']

    # filter out instances with label -1
    valid_data = valid_data.filter(lambda example: example['label'] != -1)

    # get the first val_size examples from the validation set
    val_examples = valid_data[:val_size]

    # zip the premise and hypothesis lists together to create pairs
    pairs = list(zip(val_examples['premise'], val_examples['hypothesis']))

    formatted_strings = [' </s> '.join(pair) for pair in pairs]
    result = '\n'.join(formatted_strings)

    return result, val_examples['label']

In [3]:
result, labels= get_examples("snli", 100)
print(result, labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Two women are embracing while holding to go packages. </s> The sisters are hugging goodbye while holding to go packages after just eating lunch.
Two women are embracing while holding to go packages. </s> Two woman are holding packages.
Two women are embracing while holding to go packages. </s> The men are fighting outside a deli.
Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink. </s> Two kids in numbered jerseys wash their hands.
Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink. </s> Two kids at a ballgame wash their hands.
Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink. </s> Two kids in jackets walk to school.
A man selling donuts to a customer during a

**generate prompt**

In [4]:
!pip install jinja2

In [5]:
from jinja2 import Template

In [128]:
def generate_prompt(text):
    template = Template(

        "Given 5 sentence pairs, each consisting of a Premise and a Hypothesis, separated by </s>: "
        "Assign one of the following labels to indicate the relationship: \n"
        "Contradiction: The Hypothesis is incompatible with the Premise. "
        "Example: Premise: The sky is sunny. Hypothesis: The sky is stormy. Label: Contradiction \n"
        "Entailment: The Premise logically implies the Hypothesis. The Hypothesis is a logical consequence of the Premise. "
        "Example: Premise: Children waving at camera. Hypothesis: There are children present. Label: Entailment \n"
        "Neutral: There is no clear relationship between the Premise and the Hypothesis. "
        "Example: Premise: The cat is sleeping on the couch. Hypothesis: The shelf is in the room. Label: Neutral \n"
        "The 5 sentence pairs are \n"
        " {{ text }} "
        "Please predict the labels for the sentences given above."
        "Your output should only contain the labels: Entailment, Contradiction, Neutral. \n"
        "You do not need to output the original sentence pairs given by the User, you only need to output the labels predicted by you. "
        "Your answer should contain exactly 5 labels for each of the sentence pairs given and you should place the predicted labels after 'Label:' "
        "You only need to output the labels once. "
    )
    return template.render(text=text)

In [ ]:
import os

os.environ["REPLICATE_API_TOKEN"] = "api"

In [9]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [129]:
import replicate

# function to generate LLM response
def generate_llama_response(prompt_chunk):
    pre_prompt = "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant."

    output = replicate.run('meta/llama-2-13b-chat',
                           input={"system_prompt": f"{pre_prompt}",
                                  "prompt": f"{prompt_chunk} Assistant: ",  # prompts
                                  "temperature": 0.6, "top_p": 0.8, "max_new_tokens": 100000, "repetition_penalty": 1})  # model parameters

    full_response = ""

    for item in output:
      full_response += item

    return full_response

In [130]:
# split the result string into chunks of 5 sentences each
chunk_size = 5
sentences = result.split("\n")
chunks = [sentences[i:i+chunk_size] for i in range(0, len(sentences), chunk_size)]

print(len(chunks))


20


In [131]:
all_responses = ""
for chunk in chunks:
    chunk_text = "\n".join(chunk)
    prompt_input = generate_prompt(chunk_text)

    response = generate_llama_response(prompt_input)
    all_responses += response + "\n"

print(all_responses)

Sure, I'd be happy to help! Here are the predicted labels for each sentence pair:

1. Two women are embracing while holding to go packages. </s> The sisters are hugging goodbye while holding to go packages after just eating lunch.
Label: Entailment

2. Two women are embracing while holding to go packages. </s> Two woman are holding packages.
Label: Neutral

3. Two women are embracing while holding to go packages. </s> The men are fighting outside a deli.
Label: Contradiction

4. Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink. </s> Two kids in numbered jerseys wash their hands.
Label: Entailment

5. Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink. </s> Two kids at a ballgame wash their hands.
Label: Neutral

I hope this helps! Let me know if you have any further q

**evaluate**

In [16]:
import re

In [132]:
# define a mapping of labels to numbers
label_mapping = {"Entailment": 0, "Neutral": 1, "Contradiction": 2}

# use regular expressions to extract content behind labels
pattern = re.compile(r'Label: (\w+)')
matches = pattern.findall(all_responses)

# map labels to numbers
predicted_labels = [label_mapping[label] for label in matches]

print(predicted_labels)
print(len(predicted_labels))


[0, 1, 2, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 2, 0, 0, 1, 0, 2, 0, 1, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 2, 2, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 1, 0, 0, 2, 0, 0, 1]
100


In [133]:
print(labels)

[1, 0, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 2, 0, 1, 2, 0, 2, 2, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0, 2, 1, 0, 2, 1, 0, 2, 1, 2, 1, 1, 2, 0, 1, 2, 0, 1, 1, 2, 0, 1, 2, 0, 1, 0, 2, 0, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 1, 2, 0, 1, 0, 2, 2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 2, 0, 1, 2]


In [134]:
def calculate_accuracy(predicted_labels, gold_labels):
    if len(predicted_labels) != len(gold_labels):
        raise ValueError("Lists must have the same length.")

    correct_predictions = sum(1 for pred, gold in zip(predicted_labels, gold_labels) if pred == gold)
    total_predictions = len(predicted_labels)

    accuracy = correct_predictions / total_predictions if total_predictions != 0 else 0.0
    return accuracy


In [135]:
accuracy = calculate_accuracy(predicted_labels, labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 45.00%
